In [24]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
CSV_PATH = "match_odds_wide.csv"
KEY_COLS = ["match_date", "match_time", "tournament", "match_id", "homeTeam", "awayTeam", "firstHalfHomeGoal", "firstHalfAwayGoal", "totalHomeGoal", "totalAwayGoal", "homeCorner", "awayCorner"]

In [26]:
import requests
import pandas as pd

# 1. Maç listesini çek
url_list = (
    "https://www.bilyoner.com/api/v3/mobile/aggregator/"
    "gamelist/all/v1"
)
params = {
    "tabType": 1,
    "bulletinType": 2,
    "liveEventsEnabledForPreBulletin": "true"
}
headers = {
    "User-Agent": "Mozilla/5.0"
}

resp = requests.get(url_list, params=params, headers=headers)
resp.raise_for_status()
data = resp.json()

In [27]:
event_items = [(k, v) for k, v in data.items()]
event_items = event_items[0][1]

In [28]:
event_items
match_ids = list(event_items.keys())

In [29]:
len(match_ids)

90

In [30]:
# https://www.bilyoner.com/api/v3/mobile/aggregator/match-card/2125768/odds?isLiveEvent=true&isPopular=false
from random import shuffle
shuffle(match_ids)
for match_id in match_ids[0:3]:
    # match_id = "2127178"
    match_url = f"https://www.bilyoner.com/api/v3/mobile/aggregator/match-card/{match_id}/odds?isLiveEvent=true&isPopular=false"
match_url

'https://www.bilyoner.com/api/v3/mobile/aggregator/match-card/2172147/odds?isLiveEvent=true&isPopular=false'

In [31]:
match_info = requests.get(match_url, headers=headers)
print(match_info)

<Response [200]>


In [32]:
def fetch_match_odds(match_id, is_live=True, is_popular=False):
    """
    Fetch odds JSON for a given match_id from Bilyoner API.
    Returns a pandas DataFrame with columns: market_name, odd_name, odd_value, odd_percentage.
    """
    
    league_url = f"https://www.bilyoner.com/api/v3/mobile/aggregator/gamelist/events/{match_id}"
    resp = requests.get(league_url, headers=headers)
    if resp.status_code != 200:
        print(f"Failed to fetch match {match_id}")
        return None
    data = resp.json()
    if not data:
        print(f"No data for match {match_id}")
        return None
    calendar_data = data
    # print(calendar_data)


    base_url = "https://www.bilyoner.com/api/v3/mobile/aggregator/match-card"
    params = {
        "isLiveEvent": str(is_live).lower(),
        "isPopular": str(is_popular).lower()
    }
    url = f"{base_url}/{match_id}/odds"
    resp = requests.get(url, params=params)
    if resp.status_code != 200:
        print(f"Failed to fetch match {match_id}")
        return None
    data = resp.json()
    
    home = data.get("homeTeam")
    away = data.get("awayTeam")
    markets = data.get("oddGroupTabs", [])
    # print(event_items)
    gün = calendar_data.get("esd")
    # saat = calendar_data.get("strt")
    turnuva = calendar_data.get("lgn")
    # print(gün, turnuva)


    rows = []
    for item in markets:
        if item["title"] != "Tümü":
            continue
        for market in item["matchCardOdds"]:
            mkt_name = market.get("name", "")

            if "oyuncu" in mkt_name.lower():
                continue
            if "Hangi Takım Tur Atlar" in mkt_name:
                continue
            if "kart" in mkt_name.lower():
                continue
            if "korner" in mkt_name.lower():
                continue
            if "dakikalar" in mkt_name.lower():
                continue
            if "nasıl" in mkt_name.lower():
                continue
            if "özel" in mkt_name.lower():
                continue
            if "Maçın Geri Kalanını Kim Kazanır" in mkt_name:
                continue
            if "penaltı" in mkt_name.lower():
                continue
            if "İlk Golü Hangi Takım Atar" in mkt_name:
                continue
            if "İlk Yarı Kalanını Kim Kazanır" in mkt_name:
                continue
            if "aralığ" in mkt_name.lower():
                continue

            for odd in market.get("oddList", []):
                try:
                    colname = f"{mkt_name} :: {odd['n']}"
                    value = odd["val"]
                    rows.append((colname, value))
                except:
                    continue

    if not rows:
        return None
    
    row_dict = {k: v for k, v in rows}
    
    row_dict.update({
        "match_date": gün.split("T")[0],
        "match_time": gün.split("T")[1],
        "tournament": turnuva,
        "match_id": match_id,
        "homeTeam": home,
        "awayTeam": away,
        "firstHalfHomeGoal": None,
        "firstHalfAwayGoal": None,
        "totalHomeGoal": None,
        "totalAwayGoal": None,
        "homeCorner": None,
        "awayCorner": None
    })
    # print(row_dict)
    return pd.DataFrame([row_dict])
    # columns = ["match_id", "home_team", "away_team"]

    # print(match_id, htn, atn)

    # betting_types = ["MS", "Gol", "Devre", "Korner"]
    # # print(type(markets))
    # for item in markets:
    #     if item["title"] == "Tümü":
    #         for betting_type in item["matchCardOdds"]:
    #             if "oyuncu" in betting_type["name"].lower():
    #                 continue
    #             print(betting_type["name"])
    #             for odd in betting_type["oddList"]:
    #                 try:
    #                     column_name = str.join([odd["mrn"], odd["n"]])
    #                     if column_name not in columns:
    #                         columns.append(column_name)
    #                     print(odd["mrn"], odd["n"], odd["val"])
    #                 except:
    #                     continue
    #             # print(betting_type["oddList"]["mrn"])
        
    # df = pd.DataFrame(markets)
    # return df
df_odds = fetch_match_odds(match_id)

In [33]:
import os
from tqdm import tqdm

def append_matches_to_csv(match_ids, csv_path=CSV_PATH):
    # 1) Varolan CSV'yi yükle (veya boş df), match_id sütununu string çevir
    if os.path.exists(csv_path):
        master_df = pd.read_csv(csv_path, dtype={"match_id": str})
    else:
        master_df = pd.DataFrame(columns=KEY_COLS)

    for mid in tqdm(match_ids):
        mid = str(mid)
        # print(f"Processing match_id {mid}...")
        df = fetch_match_odds(mid)
        if df is None:
            print(f"  > No odds data for match {mid}, skipping.")
            continue

        # 2) Kolon uyumu: master ve yeni df arasındaki farklı sütunları tamamla
        for col in df.columns:
            if col not in master_df.columns:
                master_df[col] = pd.NA
        for col in master_df.columns:
            if col not in df.columns:
                df[col] = pd.NA

        # 3) Satırı ekle
        master_df = pd.concat([master_df, df], ignore_index=True)

        # 4) Aynı match_id'li eski satırları sil, en son ekleneni tut
        master_df = master_df.drop_duplicates(subset=["match_id"], keep="last")

        # 5) Sütun sırasını garanti et: ilk 3 sabit, kalan alfabetik
        other_cols = [c for c in master_df.columns if c not in KEY_COLS]
        master_df = master_df[KEY_COLS + sorted(other_cols)]

        # 6) CSV’ye yaz
        master_df.to_csv(csv_path, index=False)
        # print(f"  > Written match {mid} to CSV.")

    print("All done!")

    print("All done!")

In [34]:
append_matches_to_csv(match_ids)

 43%|████▎     | 39/90 [00:39<00:47,  1.06it/s]

  > No odds data for match 2173802, skipping.


 57%|█████▋    | 51/90 [00:53<00:39,  1.02s/it]

  > No odds data for match 2178390, skipping.


100%|██████████| 90/90 [01:37<00:00,  1.09s/it]

All done!
All done!


In [35]:
import math
import pandas as pd
import requests
from tqdm import tqdm

headers = {"User-Agent": "Mozilla/5.0"}

df = pd.read_csv(CSV_PATH)

for idx, row in tqdm(df.iterrows()):
    if math.isnan(row["firstHalfHomeGoal"]) and math.isnan(row["firstHalfAwayGoal"]):
        match_id = row["match_id"]
        match_url = (
            f"https://www.bilyoner.com/api/mobile/match-card/v2/{match_id}/status"
            "?sgSportTypeId=1"
        )

        try:
            resp = requests.get(match_url, headers=headers, timeout=10)
            resp.raise_for_status()
            data = resp.json()
        except Exception as e:
            print(f"{match_id}: istek hatası → {e}")
            continue

        if "score" not in data:          # eventStatus yerine score varlığına baktık
            continue

        # -- DataFrame’in orijinalini güncelle --
        try:
            if len(data["score"]) < 3:
                continue
        except:
            continue
        
        # Score'ları map'leyerek daha güvenli çekiyoruz
        score_map = {s["scoreName"]: s for s in data["score"]}
        # print(score_map)

        try:
            df.at[idx, "firstHalfHomeGoal"] = int(score_map["SOCCER_FIRST_HALF"]["homeScore"])
            df.at[idx, "firstHalfAwayGoal"] = int(score_map["SOCCER_FIRST_HALF"]["awayScore"])
            df.at[idx, "totalHomeGoal"]     = int(score_map["SOCCER_END_SCORE"]["homeScore"])
            df.at[idx, "totalAwayGoal"]     = int(score_map["SOCCER_END_SCORE"]["awayScore"])
        except Exception as e:
            print(f"{match_id}: skor güncellenemedi → {e}")
            continue
        

# tüm döngü bitti; CSV dosyasını aynı isimle üzerine yaz
df.to_csv(CSV_PATH, index=False)
print("Güncellenmiş dosya kaydedildi.")


882it [02:05,  7.02it/s] 


Güncellenmiş dosya kaydedildi.
